In [2]:
import os
os.sys.path.append('../..')

import h5py
from IPython.display import display

import project.download_content as content

files = [
    os.path.join(content.DATAPATH, 'MODEL', '3_classes_300x300.h5_0.h5'),
    os.path.join(content.DATAPATH, 'MODEL', '3_classes_300x300.h5_1.h5'),
    os.path.join(content.DATAPATH, 'MODEL', '3_classes_300x300.h5_2.h5'),
    os.path.join(content.DATAPATH, 'MODEL', '3_classes_300x300.h5_3.h5'),
    os.path.join(content.DATAPATH, 'MODEL', '3_classes_300x300.h5_4.h5'),
    os.path.join(content.DATAPATH, 'MODEL', '3_classes_300x300.h5_5.h5'),
    os.path.join(content.DATAPATH, 'MODEL', '3_classes_300x300.h5_6.h5'),
    os.path.join(content.DATAPATH, 'MODEL', '3_classes_300x300.h5_7.h5'),
]

In [3]:
%%time
for f_path in files:
    with h5py.File(f_path, 'a') as f:
        l = list(f.keys())
        if not 'images' in l:
            unique_ids = set(i[:-2] for i in l)

            images = []

            for id in unique_ids:
                images.append([f"{id}-x".encode("ascii", "ignore"),
                               f"{id}-y".encode("ascii", "ignore")])

            f.create_dataset(name='images',
                             shape=(len(images), 2),
                             data=images,
                             dtype=h5py.special_dtype(vlen=str),
                             compression='gzip',
                             compression_opts=9)

CPU times: user 7.39 ms, sys: 0 ns, total: 7.39 ms
Wall time: 7.4 ms


In [6]:
%%time
total_batches = 0
total_images = 0

for f_path in files:
    with h5py.File(f_path, 'r') as f:
        l = list(f.keys())
        first = f['images'][0]
        last = f['images'][-1]
        partial_images = 0
        
        for x_ref, y_ref in f['images']:
            temp_count = f[x_ref].shape[0]
            partial_images += temp_count
            total_images += temp_count
            
        total_batches += len(f["images"])
        
        print('file', f_path, 'has', len(l), 'datasets')
        print('The image dataset has', len(f['images']), 'batches',
              f'({len(f["images"])} x 2) + 1 =', (len(f["images"])*2 + 1))
        print('This dataset has a total of', partial_images, 'images')
        
        print('  the first image x ref in', f_path, "is present?", 
              (first[0] in l))
        print('  the first image y ref in', f_path, "is present?", 
              (first[1] in l))
        print('   the last image x ref in', f_path, "is present?", 
              (last[0] in l))
        print('   the last image y ref in', f_path, "is present?", 
              (last[1] in l), end='\n\n')
        
        
print(f'A total of {total_batches} batches are going to be used in train')
print(f'there are {total_images} images among this batches')

file ../data/MODEL/3_classes_300x300.h5_0.h5 has 81 datasets
The image dataset has 40 batches (40 x 2) + 1 = 81
This dataset has a total of 6960 images
  the first image x ref in ../data/MODEL/3_classes_300x300.h5_0.h5 is present? True
  the first image y ref in ../data/MODEL/3_classes_300x300.h5_0.h5 is present? True
   the last image x ref in ../data/MODEL/3_classes_300x300.h5_0.h5 is present? True
   the last image y ref in ../data/MODEL/3_classes_300x300.h5_0.h5 is present? True

file ../data/MODEL/3_classes_300x300.h5_1.h5 has 81 datasets
The image dataset has 40 batches (40 x 2) + 1 = 81
This dataset has a total of 6953 images
  the first image x ref in ../data/MODEL/3_classes_300x300.h5_1.h5 is present? True
  the first image y ref in ../data/MODEL/3_classes_300x300.h5_1.h5 is present? True
   the last image x ref in ../data/MODEL/3_classes_300x300.h5_1.h5 is present? True
   the last image y ref in ../data/MODEL/3_classes_300x300.h5_1.h5 is present? True

file ../data/MODEL/3_c

In [5]:
with h5py.File(files[1], 'a') as f:
    print('''
        An example row, has N images 300x300 pixels with 3 color layers
    ''')
    display(f[f['images'][12][0]].shape)
    print('''
        and each of their targets has 8732 positions of possible bboxes
        predicting 39 classes + 1 no class and 4 numbers of anchors box
    ''')
    display(f[f['images'][12][1]].shape)


        An example row, has N images 300x300 pixels with 3 color layers
    


(175, 300, 300, 3)


        and each of their targets has 8732 positions of possible bboxes
        predicting 39 classes + 1 no class and 4 numbers of anchors box
    


(175, 8732, 8)